In [1]:
import openpyxl
import csv
import json
import requests
import re
import pandas as pd
import time
from time import sleep
import pyautogui as pag
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchAttributeException

In [2]:
options = uc.ChromeOptions()
options.headless = False
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(version_main=133)
driver.get("https://www.vinamarine.gov.vn/vi/cang-bien")

In [3]:
page_source = BeautifulSoup(driver.page_source, 'html.parser')

In [3]:
all_ports = []
while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    port_divs = soup.find_all("td", class_="views-field views-field-name")
    for port in port_divs:
        a_tag = port.find('a')
        if a_tag and a_tag.get('href'):
            href = a_tag.get('href')
            all_ports.append(f'https://www.vinamarine.gov.vn{href}')
    try:
        next_button = driver.find_element(By.CLASS_NAME, "pager-next")
        if next_button:
            next_button.click()
            time.sleep(2)  # Chờ tải trang mới
        else:
            break
    except:
        break
print(len(all_ports))
print(all_ports)

44
['https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-giang', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-ben-tre', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-binh-duong', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-binh-thuan', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-ca-na', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-can-tho', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-dung-quat', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-ha-tinh', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-hai-phong', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-hai-thinh', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-khanh-hoa', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-kien-giang', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-ky-ha', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-nam-can', 'https://www.vinamarine.gov.vn/vi/cang-bien/cang-bien-nghe', 'https://www.vinamarine.gov.

In [6]:
all_harbors = []
for port in all_ports:
    driver.get(port)
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        harbor_divs = soup.find_all("td", class_="views-field views-field-title")
        for harbor in harbor_divs:
            a_tag = harbor.find('a')
            if a_tag and a_tag.get('href'):
                href = a_tag.get('href')
                all_harbors.append(f'https://www.vinamarine.gov.vn{href}')
        try:
            next_button = driver.find_element(By.CLASS_NAME, "pager-next")
            if next_button:
                next_button.click()
                time.sleep(2)
            else:
                break
        except:
            break
print(len(all_harbors))
print(all_harbors)

263
['https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-my-thoi', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-tong-hop-binh-duong', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-phu-quy', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-tong-hop-vinh-tan', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-ca-na', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-ninh-chu', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-binh-minh', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-tong-kho-xang-dau-can-tho', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-tan-cang-cai-cui', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-x55', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-cong-ty-cp-tmdt-dau-khi-nam-song-hau-chi-nhanh-can-tho', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-chuyen-dung-phuc-thanh', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-chuyen-dung-quoc-te-lee-man', 'https://www.vinamarine.gov.vn/vi/noi-dung/ben-cang-binh-thuy

In [ ]:
def getValue(main_restaurant):
    return {
        'Name': main_restaurant.get('Name'),
        'Address': main_restaurant.get('Address'),
        'District': main_restaurant.get('District'),
        'City': main_restaurant.get('City'),
        'Latitude': main_restaurant.get('Latitude'),
        'Longitude': main_restaurant.get('Longitude')
    }

In [18]:
harbor_name_list = []
port_name_list = []
harbor_address_list = []
harbor_location_list = []

for harbor in all_harbors:
    driver.get(harbor)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Lấy tên bến cảng
    strong_tag = soup.find("h3", class_="page-header")
    if not strong_tag:
        continue  
    harbor_name = strong_tag.get_text(strip=True)
    harbor_name_list.append(harbor_name)

    # Lấy tên cảng biển
    divs = soup.find_all("div", class_="field field-name-field-pl-cang-bien field-type-taxonomy-term-reference field-label-inline clearfix")
    port_name = "NULL"  # Giá trị mặc định nếu không tìm thấy
    for div in divs:
        field_item = div.find("div", class_="field-item even")
        if field_item:
            port_name = field_item.get_text(strip=True)
            break  # Chỉ lấy một giá trị đầu tiên
    port_name_list.append(port_name)

    # Lấy địa chỉ và vị trí bến cảng
    harbor_address = "NULL"
    harbor_location = "NULL"

    rows = soup.find_all("tr")
    for row in rows:
        cells = row.find_all("td")
        if len(cells) == 2:
            label = cells[0].get_text(strip=True)
            value = cells[1].get_text(strip=True) or "NULL"

            if label == "- Địa chỉ đơn vị khai thác cảng":
                harbor_address = value
            elif label == "- Vị trí bến cảng.":
                harbor_location = value

    # Đảm bảo luôn có giá trị trong danh sách
    harbor_address_list.append(harbor_address)
    harbor_location_list.append(harbor_location)

    print("Harbor Name:", harbor_name)
    print("Port Name:", port_name)
    print("Harbor Address:", harbor_address)
    print("Harbor Location:", harbor_location)
    print("********************************************************")



Harbor Name: Bến cảng Mỹ Thới
Port Name: Cảng biển An Giang
Harbor Address: Quốc lộ 91, Mỹ Thạnh, Long Xuyên, tỉnh An Giang
Harbor Location: Quốc lộ 91, Mỹ Thạnh, Long Xuyên, tỉnh An Giang
********************************************************
Harbor Name: Bến cảng tổng hợp Bình Dương
Port Name: Cảng biển Bình Dương
Harbor Address: Ấp Ngãi Thắng, xã Bình Thắng, huyện Dĩ An, Tân Bình, Dĩ An, Bình Dương
Harbor Location: NULL
********************************************************
Harbor Name: Bến cảng Phú Quý
Port Name: Cảng biển Bình Thuận
Harbor Address: Xã Tam Thanh, huyện Phú Quý, tỉnh Bình Thuận
Harbor Location: NULL
********************************************************
Harbor Name: Bến cảng tổng hợp Vĩnh Tân
Port Name: Cảng biển Bình Thuận
Harbor Address: Tầng 6 Tòa nhà Viettel, Khu dân cư Hùng Vương, Phường Phú Thủy, Thành phố Phan Thiết, Bình Thuận
Harbor Location: NULL
********************************************************
Harbor Name: Bến cảng Cà Ná
Port Name: Cảng biển

In [20]:
df = pd.DataFrame({
    "Harbor Name": harbor_name_list,
    "Harbor Address": port_name_list,
    "Harbor Location": harbor_address_list
})
display(df)
df.to_csv("D:/TRUNG/Cảng biển/VinaMarine.csv", index= False, encoding='utf-8-sig')

,Harbor Name,Harbor Address,Harbor Location
0,Bến cảng Mỹ Thới,Cảng biển An Giang,"Quốc lộ 91, Mỹ Thạnh, Long Xuyên, tỉnh An Giang"
1,Bến cảng tổng hợp Bình Dương,Cảng biển Bình Dương,"Ấp Ngãi Thắng, xã Bình Thắng, huyện Dĩ An, Tân..."
2,Bến cảng Phú Quý,Cảng biển Bình Thuận,"Xã Tam Thanh, huyện Phú Quý, tỉnh Bình Thuận"
3,Bến cảng tổng hợp Vĩnh Tân,Cảng biển Bình Thuận,"Tầng 6 Tòa nhà Viettel, Khu dân cư Hùng Vương,..."
4,Bến cảng Cà Ná,Cảng biển Cà Ná,NULL
...,...,...,...
251,Bến cảng LPG Hồng Mộc,Cảng biển Đồng Nai,"59/2A ấp Nam Lân, Xã Bà Điểm, Huyện Hóc Môn, T..."
252,Bến cảng Vedan Phước Thái,Cảng biển Đồng Nai,"QL51, huyện Long Thành, xã Phước Thái, Đồng Nai"
253,Bến cảng Đồng Tháp,Cảng biển Đồng Tháp,"số 1551, quốc lộ 30, phường 11, thành phố Cao ..."
254,Bến cảng Sa Đéc,Cảng biển Đồng Tháp,"Khu C, khu công nghiệp Sa Đéc, phường Tân Quy ..."
